In [3]:
# source: https://www.oreilly.com/learning/algorithmic-trading-in-less-than-100-lines-of-python-code
%matplotlib inline
import configparser  # 1
import oandapy as opy  # 2
import numpy as np  # 11
import pandas as pd  # 6


In [7]:
config = configparser.ConfigParser()  # 3
config.read('oanda.cfg')  # 4

oanda = opy.API(environment='practice',
                access_token=config['oanda']['access_token'])  # 5

data = oanda.get_history(instrument='AUD_USD',  # our instrument
                         start='2016-01-08',  # start data
                         end='2016-12-10',  # end date
                         granularity='M1')  # minute bars  # 7

df = pd.DataFrame(data['candles']).set_index('time')  # 8

df.index = pd.DatetimeIndex(df.index)  # 9

df.info() # 10




df['returns'] = np.log(df['closeAsk'] / df['closeAsk'].shift(1))  # 12

cols = []  # 13

for momentum in [15, 30, 60, 120]:  # 14
    col = 'position_%s' % momentum  # 15
    df[col] = np.sign(df['returns'].rolling(momentum).mean())  # 16
    cols.append(col)  # 17



import seaborn as sns; sns.set()  # 18

strats = ['returns']  # 19

for col in cols:  # 20
    strat = 'strategy_%s' % col.split('_')[1]  # 21
    df[strat] = df[col].shift(1) * df['returns']  # 22
    strats.append(strat)  # 23

df[strats].dropna().cumsum().apply(np.exp).plot()  # 24


OandaError: OANDA API returned error code 36 (The value specified is not in the valid range: Resulting candle count is larger than maximum allowed: 5000) 